In [1]:
import numpy as np
import pandas as pd
import re
from textab import TexTab
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
wdir = os.path.join(rootdir, '_2_intermediate', 'data')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

## BDs

In [3]:
df = pd.read_csv(os.path.join(wdir,'_F_dist_religion_bch10.csv'))
df = df[['iso', 'district', 'bch10', 'major_religion', 'immg_18', 'imdwmg_18']]
#df = df[['iso', 'district', 'bch10', 'major_religion', 'immg_18', 'n_immg_18']]
df = df[((df.bch10 == 1950) | 
         (df.bch10 == 1960) |
         (df.bch10 == 1970) | 
         (df.bch10 == 1980) |
         (df.bch10 == 1990)) &
         ((df.major_religion == 'Christian') |
         (df.major_religion == 'Muslim') |
         (df.major_religion == 'Traditional'))].reset_index(drop=True)
#df = df[df.n_immg_18 >= 10].reset_index(drop=False)
df.index = [df.iso, df.district, df.bch10, df.major_religion]
df = df.drop(['iso', 'district', 'bch10', 'major_religion'], axis=1)
#df = df.drop(['iso', 'district', 'bch10', 'major_religion','n_immg_18'], axis=1)

df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()
df.columns = ['iso', 'district', 'bd', 'u_c', 'u_m', 'u_t', 'd_c', 'd_m', 'd_t']

df.index = [df.iso, df.district, df.bd]
df = df.drop(['iso', 'district', 'bd'], axis=1)
df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()

df.columns = ['iso', 'district'] + ['{}_{}'.format(var, bd) 
                                    for var in ['u_c', 'u_m', 'u_t', 'd_c', 'd_m', 'd_t'] 
                                    for bd in [1950,1960,1970,1980,1990]]

for d in ["u","d"]:
    for r in ["c","m","t"]:
        for bd in [1950,1960,1970,1980]: 
            df.loc[pd.isnull(df[f'{d}_{r}_{bd}']), f'{d}_{r}_{bd}'] = df.loc[pd.isnull(df[f'{d}_{r}_{bd}']), f'{d}_{r}_{bd+10}']
            
df1950 = df[['iso','district','u_c_1950', 'u_m_1950','u_t_1950','d_c_1950', 'd_m_1950','d_t_1950']]
df1960 = df[['iso','district','u_c_1960', 'u_m_1960','u_t_1960','d_c_1960', 'd_m_1960','d_t_1960']]
df1970 = df[['iso','district','u_c_1970', 'u_m_1970','u_t_1970','d_c_1970', 'd_m_1970','d_t_1970']]
df1980 = df[['iso','district','u_c_1980', 'u_m_1980','u_t_1980','d_c_1980', 'd_m_1980','d_t_1980']]
df1990 = df[['iso','district','u_c_1990', 'u_m_1990','u_t_1990','d_c_1990', 'd_m_1990','d_t_1990']]
df1950

,iso,district,u_c_1950,u_m_1950,u_t_1950,d_c_1950,d_m_1950,d_t_1950
0,BEN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,BEN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,BEN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,BEN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,BEN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2292,ZMB,2753.0,NaN,NaN,NaN,NaN,NaN,NaN
2293,ZMB,2754.0,NaN,NaN,NaN,NaN,NaN,NaN
2294,ZMB,2755.0,NaN,NaN,NaN,NaN,NaN,NaN
2295,ZMB,2756.0,NaN,NaN,NaN,NaN,NaN,NaN


## All birth decades

In [4]:
df = pd.read_csv(os.path.join(wdir,'_F_dist_religion_bch10.csv'))
df = df[['iso', 'district', 'bch10', 'major_religion', 'immg_18', 'n_immg_18', 'imdwmg_18', 'n_imdwmg_18']]
df = df[(~pd.isnull(df.immg_18)) | (~pd.isnull(df.imdwmg_18)) ]
df = df[df.major_religion.isin(['Christian', 'Muslim', 'Traditional'])]
df['xu'] = df.immg_18 * df.n_immg_18
df['xd'] = df.imdwmg_18 * df.n_imdwmg_18
del df['immg_18'], df['imdwmg_18'], df['bch10']
df = df[['n_immg_18', 'n_imdwmg_18', 'xu', 'xd']].groupby([df.iso, df.district, df.major_religion]).sum().reset_index(drop=False)
#df = df[df.n_immg_18 >= 10]
df['u'] = df.xu / df.n_immg_18
df['d'] = df.xd / df.n_imdwmg_18
del df['xu'], df['xd'], df['n_immg_18'], df['n_imdwmg_18']
df.index = [df.iso, df.district, df.major_religion]
del df['iso'], df['district'], df['major_religion']
df = df.unstack().reset_index(drop=False)
df.columns = df.columns.droplevel()
df.columns = ['iso', 'district', 'u_c', 'u_m', 'u_t', 'd_c', 'd_m', 'd_t']
df

,iso,district,u_c,u_m,u_t,d_c,d_m,d_t
0,BEN,1.0,0.261324,0.166667,0.086799,0.277108,0.479339,0.500000
1,BEN,2.0,0.269962,0.130326,0.057971,0.423077,0.333333,0.500000
2,BEN,3.0,0.216630,0.216003,0.106383,0.187500,0.304000,0.500000
3,BEN,4.0,0.421053,0.102079,0.000000,0.200000,0.625000,NaN
4,BEN,5.0,0.677966,0.166914,0.083333,0.302326,0.429825,NaN
...,...,...,...,...,...,...,...,...
2281,ZMB,2753.0,0.449283,NaN,0.377358,0.279251,NaN,0.357143
2282,ZMB,2754.0,0.408039,NaN,0.339286,0.347126,NaN,0.388889
2283,ZMB,2755.0,0.505353,NaN,0.428571,0.223270,0.500000,0.131579
2284,ZMB,2756.0,0.374133,0.000000,0.490323,0.293103,0.000000,0.385417


## creating sumstats

In [5]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

def sumstats(pandas_series, majrel, direction):

    ss = [majrel, direction]
    pandas_series = pandas_series.dropna()
    ss.append(pandas_series.shape[0])
    ss.append(np.round(pandas_series.mean(),3))
    ss.append(pandas_series.min())
    ss.append(np.round(np.percentile(pandas_series, 10),3))
    ss.append(np.round(np.percentile(pandas_series, 50),3))
    ss.append(np.round(np.percentile(pandas_series, 90),3))
    ss.append(np.round(pandas_series.max(),3))
    return ss
    

def make_sumstats_df(df):
    
    rows = []
    rels = ['Christian', 'Muslim', 'Traditional'] * 2
    dirs = ['up'] * 3 + ['down'] * 3
    for ic, col in enumerate(df.columns[2:]):
        majrel = rels[ic]
        direction = dirs[ic]
        
        rows.append(sumstats(df[col], majrel, direction))
        
    df = pd.DataFrame(rows, columns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max'])
    return df

In [6]:
dfall = make_sumstats_df(df)
df1950 = make_sumstats_df(df1950)
df1960 = make_sumstats_df(df1960)
df1970 = make_sumstats_df(df1970)
df1980 = make_sumstats_df(df1980)
df1990 = make_sumstats_df(df1990)

## writing .tex tables

### all birth decades

In [7]:
tt = TexTab(dfall)

In [8]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gaps = ['']
cap = "Summary statistics, district-level IM, ages 14-18, all birth decades"
#cap = "Summary statistics, district-level IM, ages 14-18, all birth decades, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_allbch"
#lab = "tab:_sumstats_im_dist_majrel_1418_allbch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [9]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [10]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_allbch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_allbch_districts.tex", "w")
fh.write(tab)
fh.close()

### 1950s

In [11]:
tt = TexTab(df1950)

In [12]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gaps = ['']
cap = "Summary statistics, district-level IM, ages 14-18, 1950s"
#cap = "Summary statistics, district-level IM, ages 14-18, 1950s, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_1950bch"
#lab = "tab:_sumstats_im_dist_majrel_1418_1950bch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [13]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [14]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1950bch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1950bch_districts.tex", "w")
fh.write(tab)
fh.close()

### 1960s

In [15]:
tt = TexTab(df1960)

In [16]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gap = ['']
cap = "Summary statistics, district-level IM, ages 14-18, 1960s"
#cap = "Summary statistics, district-level IM, ages 14-18, 1960s, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_1960bch"
#lab = "tab:_sumstats_im_dist_majrel_1418_1960bch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [17]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [18]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1960bch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1960bch_districts.tex", "w")
fh.write(tab)
fh.close()

### 1970s

In [19]:
tt = TexTab(df1970)

In [20]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gaps = ['']
cap = "Summary statistics, district-level IM, ages 14-18, 1970s"
#cap = "Summary statistics, district-level IM, ages 14-18, 1970s, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_1970bch"
#lab = "tab:_sumstats_im_dist_majrel_1418_1970bch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [21]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [22]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1970bch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1970bch_districts.tex", "w")
fh.write(tab)
fh.close()

### 1980s

In [23]:
tt = TexTab(df1980)

In [24]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gaps = ['']
cap = "Summary statistics, district-level IM, ages 14-18, 1980s"
#cap = "Summary statistics, district-level IM, ages 14-18, 1980s, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_1980bch"
#lab = "tab:_sumstats_im_dist_majrel_1418_1980bch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [25]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [26]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1980bch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1980bch_districts.tex", "w")
fh.write(tab)
fh.close()

### 1990s

In [27]:
tt = TexTab(df1990)

In [28]:
cns = ['religion', 'direction', 'N', 'mean', 'min', 'p10', 'p50', 'p90', 'max']
lcols = ['religion', 'direction']
gap = ['']
cap = "Summary statistics, district-level IM, ages 14-18, 1990s"
#cap = "Summary statistics, district-level IM, ages 14-18, 1990s, less than 10 districts"
lab = "tab:_sumstats_im_dist_majrel_1418_1990bch"
#lab = "tab:_sumstats_im_dist_majrel_1418_1990bch_lt10d"
lines = []
notes = '''TABLES NOTES.
'''

In [29]:
tab = tt.create_tab_onepanel(cns=cns, gaps=gaps, lcols=lcols,
                             cap=cap, lab=lab, lines=lines, notes=notes, clines=False)
tab = re.sub('\n&&\(1\).+\\\\\n\\\\hline', '', tab)
tab = re.sub('\\\\columnwidth', '0.6\\\\columnwidth', tab)

In [30]:
fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1990bch.tex", "w")
#fh = open(outdir + "/_2_sumstats_dist_im_majrel_1418_1990bch_districts.tex", "w")
fh.write(tab)
fh.close()